In [586]:
import torch
import models,settings
import numpy as np
import sklearn.cluster as scikit_cluster
import pickle
import pandas as pd

In [630]:
with open("TestRun/OriginalActivations.pkl", 'rb') as f:
    final_activs=pickle.load(f)
with open("Sentences.pkl", 'rb') as f:
    sents=pickle.load(f)


In [631]:
final_activs = np.transpose(np.array(final_activs))


In [628]:
final_activs[6]

array([0.       , 0.       , 4.7760525, ..., 0.       , 0.       ,
       0.       ], dtype=float32)

In [552]:
def cluster_activ_ranges(num_clusters, neuron, final_actives):
    clustering=scikit_cluster.KMeans(n_clusters= num_clusters, random_state=1234).fit(final_actives[neuron].reshape(-1,1))
    activation_range= {}
    for index, label in enumerate(clustering.labels_):
        if label in activation_range.keys():
            activation_range[label].append(final_actives[neuron][index])
        else:
            activation_range[label] = [final_actives[neuron][index]]
        activation_range[label].sort()
    return sort_dict(activation_range)

In [553]:
def sort_dict(activation_range):
    ranges = []
    activation_range_ = activation_range.values()
    activation_range_= sorted(activation_range_, key=lambda item: item[0])
    for i in activation_range_:
        ranges.append((min(i), max(i)))
    return ranges

In [663]:
def find_explaining_neurons(cluster):
    df = pd.read_csv(f"TestRun/Cluster{cluster}IOUS1024N.csv")
    return df['unit'].tolist()

In [658]:
775 in find_explaining_neurons(1)

False

In [662]:
def get_samples_for_neuron(num_clusters, final_activations, neuron, ranges):
    pairs = [] #dict of sets
    for cluster in range(num_clusters):
        pairs_ = []
        for sample, activation in enumerate(final_activations[neuron]):
            if activation >= ranges[cluster][0] and activation <= ranges[cluster][1]:
                pairs_.append(sample)
        pairs.append(pairs_)
    return pairs

In [664]:
def get_formula_for_neuron(cluster, neuron):
    df = pd.read_csv(f"TestRun/Cluster{cluster}IOUS1024N.csv")
    return df[df['unit']==neuron].best_name.item()

In [644]:
get_formula_for_neuron(1, 775)

Neurn  Empty DataFrame
Columns: [unit, best_name, best_iou]
Index: []


ValueError: can only convert an array of size 1 to a Python scalar

In [618]:
def get_sentences(pairs_indices, sentences):
    for i, pairs in enumerate(pairs_indices):
        for pair in pairs[:2]:
            yield i+1, pair, sentences[2*pair], sentences[(2*pair) + 1]


In [619]:
import os,csv
def write_file(file,fields,data):
    if os.path.exists(file):
        with open(file, 'a') as csvfile:
            writer = csv.writer(csvfile, delimiter=',')
            writer.writerows(data)
    else:
        with open(file, 'w') as csvfile:
            writer = csv.writer(csvfile, delimiter=',')
            writer.writerow(fields)
            writer.writerows(data)

In [ ]:
def cluster_activ_ranges(num_clusters, neuron, final_actives):
    clustering=scikit_cluster.KMeans(n_clusters= num_clusters, random_state=1234).fit(final_actives[neuron].reshape(-1,1))
    activation_range= {}
    for index, label in enumerate(clustering.labels_):
        if label in activation_range.keys():
            activation_range[label].append(final_actives[neuron][index])
        else:
            activation_range[label] = [final_actives[neuron][index]]
        activation_range[label].sort()
    return sort_dict(activation_range)
def sort_dict(activation_range):
    ranges = []
    activation_range_ = activation_range.values()
    activation_range_= sorted(activation_range_, key=lambda item: item[0])
    for i in activation_range_:
        ranges.append((min(i), max(i)))
    return ranges
def find_explaining_neurons(cluster):
    df = pd.read_csv(f"TestRun/Cluster{cluster}IOUS1024N.csv")
    return df['unit'].tolist()
def get_samples_for_neuron(num_clusters, final_activations, neuron, ranges):
    pairs = [] #dict of sets
    for cluster in range(num_clusters):
        pairs_ = []
        for sample, activation in enumerate(final_activations[neuron]):
            if activation >= ranges[cluster][0] and activation <= ranges[cluster][1]:
                pairs_.append(sample)
        pairs.append(pairs_)
    return pairs
def get_formula_for_neuron(cluster, neuron):
    df = pd.read_csv(f"TestRun/Cluster{cluster}IOUS1024N.csv")
    return df[df['unit']==neuron].best_name.item()
def get_sentences(pairs_indices, sentences):
    for i, pairs in enumerate(pairs_indices):
        for pair in pairs[:2]:
            yield i+1, pair, sentences[2*pair], sentences[(2*pair) + 1]
import os,csv
def write_file(file,fields,data):
    if os.path.exists(file):
        with open(file, 'a') as csvfile:
            writer = csv.writer(csvfile, delimiter=',')
            writer.writerows(data)
    else:
        with open(file, 'w') as csvfile:
            writer = csv.writer(csvfile, delimiter=',')
            writer.writerow(fields)
            writer.writerows(data)
num_clusters=[3,4,5,10]
import random
neurons=find_explaining_neurons(cluster=1)
neurons=random.sample(neurons, k=10)
for num_cluster in num_clusters:
    print(f"Working on {num_cluster} clusters")
    for neuron in neurons:
        print("Neuron: ", neuron)
        ranges = cluster_activ_ranges(num_clusters=num_cluster, neuron=neuron, final_actives=final_activs)
        pairs_indices=get_samples_for_neuron(num_clusters=num_cluster, final_activations=final_activs, neuron=neuron, ranges=ranges)
        field=['neuron','cluster','formula', 'pair_num', 'pre', 'hyp']
        file=f'TestRun/Results{num_cluster}Cluster.csv'
        for cluster, pair_num, p, h in get_sentences(pairs_indices, sents):
            formula = get_formula_for_neuron(cluster, neuron)
            write_file(file,field,[[neuron, cluster, formula, pair_num, p, h]])

In [667]:
num_clusters=[3,4,5,10]
import random
neurons=find_explaining_neurons(cluster=1)
neurons=random.sample(neurons, k=10)
for num_cluster in num_clusters:
    print(f"Working on {num_cluster} clusters")
    for neuron in neurons:
        print("Neuron: ", neuron)
        ranges = cluster_activ_ranges(num_clusters=num_cluster, neuron=neuron, final_actives=final_activs)
        pairs_indices=get_samples_for_neuron(num_clusters=num_cluster, final_activations=final_activs, neuron=neuron, ranges=ranges)
        field=['neuron','cluster','formula', 'pair_num', 'pre', 'hyp']
        file=f'TestRun/Results{num_cluster}Cluster.csv'
        for cluster, pair_num, p, h in get_sentences(pairs_indices, sents):
            formula = get_formula_for_neuron(cluster, neuron)
            write_file(file,field,[[neuron, cluster, formula, pair_num, p, h]])

Working on 5 clusters
Neuron:  390


KeyboardInterrupt: 

In [584]:
8192 in pairs_indices[2]

True

In [451]:
l=list(sam)
print(len(l))
#for i in l:
 #   print(i, sents[2*i], sents[(2*i)+1])

405


In [ ]:
clus4=[]
for l, a in zip(c.labels_, final_activs[6]):
    if l==1:
        clus4.append(a.item())
print(clus4)
sam=set()
for a in clus4:
    for i in np.where(final_activs[6]>a)[0]:
        sam.add(i)
sam
    

In [318]:
final_weight=torch.randn(1024,2048)
final_weight.shape, final_weight

(torch.Size([1024, 2048]),
 tensor([[ 0.1208, -0.7709,  1.4369,  ..., -0.8955, -1.2836, -0.1834],
         [-0.6939,  0.7449, -0.5268,  ...,  0.1234,  0.3660,  0.1675],
         [-0.2353,  2.4709,  1.5476,  ..., -0.1652, -0.8350, -1.2371],
         ...,
         [-1.1260, -0.8813, -0.0670,  ...,  0.5094, -0.6077, -0.0805],
         [ 0.2784,  0.0794, -1.5159,  ..., -0.1675,  0.5769,  0.2123],
         [-1.5695,  0.1398,  0.7238,  ...,  0.2344, -1.0602, -0.0582]]))

In [319]:
final_weight.sum()

tensor(1396.9832)

In [320]:
final_weight_t=final_weight.t()
mask=torch.ones(final_weight_t.shape)
sorted_weights = np.sort(np.abs(final_weight_t[mask == 1]))
print(sorted_weights)
# Determine the cutoff for weights to be pruned.

cutoff_index = np.round(0.05 * sorted_weights.size).astype(int)
print(cutoff_index)
cutoff = sorted_weights[cutoff_index - 1] 
# Prune all weights below the cutoff.

new_mask= torch.where(torch.abs(torch.tensor(final_weight_t)) <= cutoff, torch.zeros(mask.shape), mask)
new_weights= torch.where(torch.abs(torch.tensor(final_weight_t)) <= cutoff, torch.zeros(mask.shape), torch.tensor(final_weight_t))
new_mask, new_weights.shape


[0.0000000e+00 0.0000000e+00 3.1314639e-07 ... 4.9532766e+00 4.9900460e+00
 5.0430379e+00]
104858


/tmp/ipykernel_827/2280124660.py:12: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  new_mask= torch.where(torch.abs(torch.tensor(final_weight_t)) <= cutoff, torch.zeros(mask.shape), mask)
/tmp/ipykernel_827/2280124660.py:13: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  new_weights= torch.where(torch.abs(torch.tensor(final_weight_t)) <= cutoff, torch.zeros(mask.shape), torch.tensor(final_weight_t))


(tensor([[1., 1., 1.,  ..., 1., 1., 1.],
         [1., 1., 1.,  ..., 1., 1., 1.],
         [1., 1., 1.,  ..., 1., 1., 1.],
         ...,
         [1., 1., 1.,  ..., 1., 1., 1.],
         [1., 1., 1.,  ..., 1., 1., 1.],
         [1., 1., 1.,  ..., 1., 1., 0.]]),
 torch.Size([2048, 1024]))

In [321]:

mask=torch.ones(final_weight.shape)
sorted_weights = np.sort(np.abs(final_weight[mask == 1]))
print(sorted_weights)
# Determine the cutoff for weights to be pruned.

cutoff_index = np.round(0.05 * sorted_weights.size).astype(int)
print(cutoff_index)
cutoff = sorted_weights[cutoff_index - 1] 
# Prune all weights below the cutoff.

new_mask= torch.where(torch.abs(torch.tensor(final_weight)) <= cutoff, torch.zeros(mask.shape), mask)
new_weights_= torch.where(torch.abs(torch.tensor(final_weight)) <= cutoff, torch.zeros(mask.shape), torch.tensor(final_weight))
new_mask, new_weights_.shape


[0.0000000e+00 0.0000000e+00 3.1314639e-07 ... 4.9532766e+00 4.9900460e+00
 5.0430379e+00]
104858


/tmp/ipykernel_827/179194577.py:11: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  new_mask= torch.where(torch.abs(torch.tensor(final_weight)) <= cutoff, torch.zeros(mask.shape), mask)
/tmp/ipykernel_827/179194577.py:12: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  new_weights_= torch.where(torch.abs(torch.tensor(final_weight)) <= cutoff, torch.zeros(mask.shape), torch.tensor(final_weight))


(tensor([[1., 1., 1.,  ..., 1., 1., 1.],
         [1., 1., 1.,  ..., 1., 1., 1.],
         [1., 1., 1.,  ..., 1., 1., 1.],
         ...,
         [1., 1., 1.,  ..., 1., 1., 1.],
         [1., 1., 1.,  ..., 1., 1., 1.],
         [1., 1., 1.,  ..., 1., 1., 0.]]),
 torch.Size([1024, 2048]))

In [323]:
assert torch.equal(new_weights_.t(),new_weights)

In [253]:
import pickle
# Read the list of lists from the pickle file
with open("weights_postcopy.pkl", 'rb') as f:
    weights_postcopy=pickle.load(f)
with open("weights_precopy.pkl", 'rb') as f:
    weights_pre_copy=pickle.load(f)
with open("dead_neurons.pkl", 'rb') as f:
    d=pickle.load(f)
len(d)

637

In [259]:
for i,w in enumerate(weights_postcopy):
    if w.sum() == 0:
        assert i in d
    else:
        assert not i in d

In [256]:
for i,w in enumerate(weights_pre_copy):
    if i in d:
        pr

tensor([[ 0.1021,  0.1023,  0.0403,  ...,  0.2561,  0.0333,  0.1069],
        [ 0.0402,  0.0016, -0.0737,  ...,  0.0378,  0.0548, -0.0057],
        [-0.1801,  0.1012,  0.1627,  ...,  0.0704, -0.1102, -0.0075],
        ...,
        [-0.0222,  0.0016, -0.0726,  ...,  0.0095, -0.1076, -0.0332],
        [-0.0225,  0.0399,  0.0214,  ...,  0.0184, -0.0385, -0.1041],
        [ 0.1054, -0.0383,  0.0160,  ..., -0.0532, -0.4026, -0.0549]],
       device='cuda:0')

In [260]:
weights_postcopy

tensor([[ 0.1021,  0.1023,  0.0403,  ...,  0.2561,  0.0333,  0.1069],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        ...,
        [-0.0222,  0.0016, -0.0726,  ...,  0.0095, -0.1076, -0.0332],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.1054, -0.0383,  0.0160,  ..., -0.0532, -0.4026, -0.0549]],
       device='cuda:0')

In [227]:
listt=[1, 2, 3, 4, 5, 7, 9, 10, 11, 12, 13, 14, 18, 20, 21, 23, 24, 25, 26, 27, 29, 30, 31, 32, 33, 34, 35, 36, 38, 42, 43, 44, 46, 47, 48, 49, 51, 53, 54, 55, 56, 57, 58, 60, 62, 64, 66, 68, 69, 72, 73, 74, 75, 78, 79, 80, 81, 82, 84, 88, 90, 91, 93, 94, 95, 96, 97, 100, 101, 102, 103, 104, 105, 107, 108, 109, 110, 111, 112, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 130, 131, 135, 136, 138, 141, 142, 143, 144, 145, 146, 147, 148, 149, 152, 153, 154, 155, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 172, 174, 175, 177, 178, 180, 182, 183, 184, 186, 187, 188, 189, 190, 193, 194, 195, 196, 199, 200, 201, 202, 205, 206, 207, 208, 210, 211, 212, 214, 216, 217, 218, 219, 220, 221, 222, 223, 224, 225, 226, 227, 228, 229, 233, 235, 236, 237, 240, 242, 243, 245, 246, 249, 250, 251, 252, 255, 256, 258, 259, 263, 264, 266, 268, 269, 271, 272, 273, 274, 276, 277, 278, 279, 281, 282, 286, 289, 291, 295, 299, 300, 301, 303, 305, 306, 310, 311, 313, 314, 315, 317, 318, 319, 321, 322, 323, 324, 325, 327, 329, 330, 331, 333, 334, 335, 337, 338, 339, 340, 342, 343, 344, 345, 347, 349, 350, 351, 352, 353, 354, 355, 356, 357, 358, 360, 361, 363, 364, 365, 366, 370, 371, 372, 373, 374, 379, 381, 383, 384, 388, 393, 394, 396, 397, 398, 401, 402, 405, 406, 407, 408, 409, 414, 415, 417, 418, 419, 421, 422, 425, 426, 427, 428, 430, 431, 432, 433, 438, 439, 440, 441, 442, 443, 444, 446, 447, 449, 451, 452, 453, 455, 456, 459, 460, 461, 464, 465, 467, 468, 469, 471, 472, 474, 475, 476, 477, 479, 481, 482, 484, 485, 488, 489, 490, 492, 493, 494, 496, 503, 504, 505, 508, 509, 513, 518, 519, 520, 521, 526, 529, 530, 531, 532, 535, 538, 539, 541, 542, 544, 545, 546, 547, 548, 550, 551, 552, 554, 555, 556, 558, 559, 560, 561, 562, 564, 567, 568, 569, 570, 571, 573, 574, 576, 580, 581, 582, 583, 585, 586, 587, 589, 591, 592, 594, 595, 596, 597, 598, 599, 600, 603, 607, 608, 610, 611, 612, 613, 616, 617, 621, 622, 624, 625, 626, 627, 628, 629, 630, 632, 633, 634, 635, 637, 638, 639, 641, 642, 644, 645, 646, 648, 649, 650, 651, 653, 654, 655, 657, 659, 660, 661, 663, 664, 665, 666, 667, 668, 669, 670, 671, 673, 677, 678, 680, 681, 682, 683, 684, 686, 687, 688, 690, 692, 693, 694, 695, 696, 697, 701, 702, 704, 706, 707, 708, 709, 711, 712, 713, 714, 717, 718, 720, 721, 722, 724, 726, 727, 728, 730, 731, 734, 735, 736, 737, 739, 740, 746, 747, 748, 749, 751, 753, 754, 755, 756, 757, 758, 759, 760, 761, 762, 763, 764, 766, 767, 768, 769, 770, 772, 774, 777, 778, 779, 780, 781, 783, 784, 785, 786, 788, 789, 790, 792, 794, 795, 797, 799, 801, 802, 803, 804, 806, 807, 808, 809, 810, 811, 812, 814, 815, 816, 817, 818, 819, 822, 823, 824, 825, 826, 827, 828, 829, 830, 831, 832, 834, 835, 836, 838, 839, 840, 841, 843, 844, 845, 846, 847, 848, 849, 851, 852, 853, 855, 856, 857, 860, 862, 863, 866, 867, 868, 869, 871, 872, 873, 876, 877, 878, 879, 880, 881, 883, 884, 887, 888, 889, 890, 891, 893, 894, 896, 897, 898, 899, 900, 901, 902, 903, 904, 906, 909, 910, 911, 912, 914, 915, 916, 918, 920, 921, 923, 927, 928, 929, 930, 931, 933, 935, 937, 938, 940, 942, 943, 944, 945, 946, 948, 950, 951, 952, 953, 954, 955, 956, 958, 960, 961, 962, 964, 965, 966, 967, 968, 969, 971, 973, 974, 975, 976, 978, 981, 984, 985, 987, 988, 989, 991, 992, 993, 994, 995, 996, 997, 999, 1001, 1002, 1003, 1004, 1005, 1006, 1009, 1010, 1011, 1012, 1013, 1014, 1015, 1016, 1017, 1018, 1019, 1022] 
for i in range(1024):
    if i  in listt:
        assert torch.load("Cluster4masks.pt")[i].sum()==0
        assert torch.load("Cluster3masks.pt")[i].sum()==0
        assert torch.load("Cluster2masks.pt")[i].sum()==0
        assert torch.load("Cluster1masks.pt")[i].sum()==0, i

## b=torch.randn(10)
b=b.reshape(1,-1)
print(b.shape)
clusters = scikit_cluster.KMeans(n_clusters= 5, random_state=1234).fit(b.t())
clusters.labels_ , b.t()

In [190]:
ones=0
twos=0
threes=0
fours=0
fives=0
for i in clusters.labels_:
    if i==1:
        ones+=1
    elif i==2:
        twos+=1
    elif i==3:
        threes+=1
    elif i==4:
        fours+=1
    else:
        fives+=1

In [191]:
ones, twos, threes, fours, fives

(2, 4, 1, 1, 2)

In [180]:
clusters = scikit_cluster.KMeans(n_clusters= 5, random_state=1234).fit(b.t())

In [ ]:
 scikit_cluster.KMeans(n_clusters= 5, random_state=1234).fit(b.t())

In [181]:
clusters.labels_ , b.t()

(array([3, 0, 2, 0, 2, 4, 4, 1, 4, 0], dtype=int32),
 tensor([[-2.1231],
         [ 0.2412],
         [-0.5005],
         [ 0.7508],
         [-0.8402],
         [-0.0667],
         [-0.2084],
         [-1.3324],
         [-0.1226],
         [ 0.3313]]))

In [173]:
import torch
c1="IncMasks/Masks0.0%Pruned/Cluster1masks.pt"
m0c1=torch.load(c1)
torch.masked_select(m0w,m0c1)

RuntimeError: The size of tensor a (1024) must match the size of tensor b (2048) at non-singleton dimension 1

In [7]:
import torch,settings,models
ckpt = torch.load(f"../{settings.MODEL}", map_location="cpu")
clf = models.BowmanEntailmentClassifier
enc = models.TextEncoder(len(ckpt["stoi"]))
model=clf(enc)
model.load_state_dict(ckpt['state_dict'])
model.mlp[0].weight.t().detach().cpu().numpy().shape

(2048, 1024)

In [40]:
n=torch.randn(2,4)
print(n)
def prune_by_percent_once(percent, mask, final_weight):
    # Put the weights that aren't masked out in sorted order.
    sorted_weights = np.sort(np.abs(final_weight[mask != 0]))

    # Determine the cutoff for weights to be pruned.
    
    cutoff_index = np.round(percent * sorted_weights.size).astype(int)
    cutoff = sorted_weights[cutoff_index - 1] 
    # Prune all weights below the cutoff.
    new_mask= torch.where(torch.abs(torch.tensor(final_weight)) <= cutoff, torch.zeros(mask.shape), mask)
    
    new_weights= torch.where(torch.abs(torch.tensor(final_weight)) <= cutoff, torch.zeros(mask.shape), torch.tensor(final_weight))
    return new_mask, new_weights
prune_by_percent_once(0.5, torch.ones(weights.shape),weights) 

tensor([[-0.6631,  1.0282,  0.0766, -0.3954],
        [-1.7383, -0.3737, -0.1654,  1.2035]])


(tensor([[1., 0., 1.,  ..., 0., 0., 1.],
         [1., 0., 1.,  ..., 0., 0., 0.],
         [0., 1., 1.,  ..., 1., 0., 0.],
         ...,
         [1., 0., 1.,  ..., 0., 0., 0.],
         [0., 0., 1.,  ..., 1., 0., 1.],
         [1., 0., 0.,  ..., 0., 1., 1.]]),
 tensor([[ 0.1021,  0.0000, -0.1801,  ...,  0.0000,  0.0000,  0.1054],
         [ 0.1023,  0.0000,  0.1012,  ...,  0.0000,  0.0000,  0.0000],
         [ 0.0000, -0.0737,  0.1627,  ..., -0.0726,  0.0000,  0.0000],
         ...,
         [ 0.2561,  0.0000,  0.0704,  ...,  0.0000,  0.0000,  0.0000],
         [ 0.0000,  0.0000, -0.1102,  ..., -0.1076,  0.0000, -0.4026],
         [ 0.1069,  0.0000,  0.0000,  ...,  0.0000, -0.1041, -0.0549]]))

In [73]:
prune_metrics_dir = "../models/snli/Inc/prune_metrics/0.5%Pruned/model_best.pth"
weights=torch.load(f"{prune_metrics_dir}")['state_dict']
model.load_state_dict(weights)

<All keys matched successfully>

In [74]:
a=torch.where(model.mlp[0].weight==0,1,0).sum()/100
a.item()

54.54999923706055

In [246]:
weights = model.mlp[0].weight.t().detach().cpu().numpy()
weights[d] = np.zeros((1,1024))

In [247]:
weights

array([[ 0.1020597 ,  0.04020922, -0.18008865, ..., -0.02218907,
        -0.02251127,  0.10539793],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       ...,
       [ 0.25605595,  0.03776345,  0.07038172, ...,  0.00953927,
         0.01841197, -0.0531592 ],
       [ 0.0333262 ,  0.05475619, -0.11017767, ..., -0.10764154,
        -0.03854869, -0.40262905],
       [ 0.1068556 , -0.00570201, -0.00749541, ..., -0.03318856,
        -0.10414948, -0.05488989]], dtype=float32)

In [76]:
nm,nw=prune_by_percent_once(0.5, torch.ones(weights.shape),weights) 

In [130]:
mask=torch.ones(weights.shape)
final_weight=np.sort(np.abs(weights>0)).astype('int')

In [81]:
#model.mlp[0].weight.t().detach().cpu().copy_(nw)
print(torch.where(model.mlp[0].weight.t().detach().cpu() != nw,1,0).sum())

tensor(0)


In [55]:
a=torch.where(torch.abs(torch.tensor(final_weight)) == 1, torch.zeros(mask.shape), mask)

NameError: name 'final_weight' is not defined

In [65]:
torch.equal(model.mlp[0].weight.t().detach().cpu(),nw)

True

In [129]:
np.round((torch.where(model.mlp[0].weight.t()==0,1,0).sum()/(1024*2048)),1)

tensor(0.5000)

In [86]:
import formula as FM
l=[1,2,3]
FM.Or(4,5)

OR(4, 5)